In [1]:
import sys
sys.path.append('../')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import utilities

# begin by opening and examining the SP500 companies


tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers["Symbol"].values)

/tmp/ipykernel_29831/1693632718.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


['MMM' 'AOS' 'ABT' 'ABBV' 'ACN' 'ADBE' 'AMD' 'AES' 'AFL' 'A' 'APD' 'ABNB'
 'AKAM' 'ALB' 'ARE' 'ALGN' 'ALLE' 'LNT' 'ALL' 'GOOGL' 'GOOG' 'MO' 'AMZN'
 'AMCR' 'AEE' 'AAL' 'AEP' 'AXP' 'AIG' 'AMT' 'AWK' 'AMP' 'AME' 'AMGN' 'APH'
 'ADI' 'ANSS' 'AON' 'APA' 'AAPL' 'AMAT' 'APTV' 'ACGL' 'ADM' 'ANET' 'AJG'
 'AIZ' 'T' 'ATO' 'ADSK' 'ADP' 'AZO' 'AVB' 'AVY' 'AXON' 'BKR' 'BALL' 'BAC'
 'BK' 'BBWI' 'BAX' 'BDX' 'BRK.B' 'BBY' 'BIO' 'TECH' 'BIIB' 'BLK' 'BX' 'BA'
 'BKNG' 'BWA' 'BXP' 'BSX' 'BMY' 'AVGO' 'BR' 'BRO' 'BF.B' 'BLDR' 'BG'
 'CDNS' 'CZR' 'CPT' 'CPB' 'COF' 'CAH' 'KMX' 'CCL' 'CARR' 'CTLT' 'CAT'
 'CBOE' 'CBRE' 'CDW' 'CE' 'COR' 'CNC' 'CNP' 'CDAY' 'CF' 'CHRW' 'CRL'
 'SCHW' 'CHTR' 'CVX' 'CMG' 'CB' 'CHD' 'CI' 'CINF' 'CTAS' 'CSCO' 'C' 'CFG'
 'CLX' 'CME' 'CMS' 'KO' 'CTSH' 'CL' 'CMCSA' 'CMA' 'CAG' 'COP' 'ED' 'STZ'
 'CEG' 'COO' 'CPRT' 'GLW' 'CTVA' 'CSGP' 'COST' 'CTRA' 'CCI' 'CSX' 'CMI'
 'CVS' 'DHR' 'DRI' 'DVA' 'DE' 'DAL' 'XRAY' 'DVN' 'DXCM' 'FANG' 'DLR' 'DFS'
 'DG' 'DLTR' 'D' 'DPZ' 'DOV' 'DOW' 'DHI' 'DTE' 'DUK' '

In [2]:
def retrieve_data(ticker):
    filename = "../data/stocks/{}_full_1min_adjsplit.txt".format(ticker)

    try:
        df = pd.read_csv(filename, sep=",", header=0)
    except FileNotFoundError:
        print("{}: file not found".format(filename))
        return None

    df.columns = ["timestamp", "open", "high", "low", "close", "volume"]
    df["ticker"] = ticker
    df = utilities.timestamp_to_date_and_time(df)
    
    misses = utilities.missing_markethours(df)
    
    print(ticker, misses)
    return misses

In [3]:
all_misses = pd.DataFrame(columns=["ticker", "miss_rate"])
tickers = tickers["Symbol"].values

for ticker in tickers:
    misses = retrieve_data(ticker)
    new_row = pd.DataFrame({"ticker": ticker, "miss_rate": [misses]})
    all_misses = pd.concat([all_misses, new_row])

../data/stocks/Symbol_full_1min_adjsplit.txt: file not found
../data/stocks/Security_full_1min_adjsplit.txt: file not found
../data/stocks/GICS Sector_full_1min_adjsplit.txt: file not found
../data/stocks/GICS Sub-Industry_full_1min_adjsplit.txt: file not found
../data/stocks/Headquarters Location_full_1min_adjsplit.txt: file not found
../data/stocks/Date added_full_1min_adjsplit.txt: file not found
CIK 0.7012269191171543
../data/stocks/Founded_full_1min_adjsplit.txt: file not found


In [4]:
all_misses.to_csv("misses.csv", index=False)